In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.options.display.max_rows = 999
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

from xgboost import XGBRegressor


In [2]:
FeatureTrain = pd.read_csv('./data/dengue_features_train.csv',parse_dates = ['week_start_date'])
TargetTrain = pd.read_csv('./data/dengue_labels_train.csv' )
FeatureTest = pd.read_csv('./data/dengue_features_test.csv',parse_dates = ['week_start_date'])
Answersheet = pd.read_csv('./data/submission_format.csv' )

### look at target first

In [3]:
"""
This is the difference between two measure

https://journals.ametsoc.org/doi/pdf/10.1175/BAMS-D-14-00226.1


"""

'\nThis is the difference between two measure\n\nhttps://journals.ametsoc.org/doi/pdf/10.1175/BAMS-D-14-00226.1\n\n\n'

In [4]:
panel = TargetTrain.merge(FeatureTrain,on = ['city','year','weekofyear'] ,how = 'outer') 
 
kelvin = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k',   'reanalysis_max_air_temp_k','reanalysis_min_air_temp_k','reanalysis_dew_point_temp_k']
panel.loc[:,kelvin] = panel.loc[:,kelvin]-273.15# kelvin to C
FeatureTest.loc[:,kelvin] =FeatureTest.loc[:,kelvin]-273.15# kelvin to C
panel.columns

timeid = ['year', 'weekofyear']
green = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']
precipitation  =['precipitation_amt_mm','reanalysis_sat_precip_amt_mm','station_precip_mm','reanalysis_precip_amt_kg_per_m2',]
#precipitation = ['precipitation_amt_mm']
avg_temp = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k',  'station_avg_temp_c' ]
#avg_temp =  [   'station_avg_temp_c' ]     
min_temp = ['station_min_temp_c','reanalysis_min_air_temp_k']
#min_temp = ['station_min_temp_c' ]
max_temp = ['station_max_temp_c','reanalysis_max_air_temp_k']
#max_temp = ['station_max_temp_c']
dtr =  ['reanalysis_tdtr_k', 'station_diur_temp_rng_c']
#dtr =  [  'station_diur_temp_rng_c']

humid = ['reanalysis_dew_point_temp_k','reanalysis_specific_humidity_g_per_kg', 'reanalysis_relative_humidity_percent']
#humid = ['reanalysis_dew_point_temp_k','reanalysis_specific_humidity_g_per_kg']
 
selected = humid + dtr + max_temp +min_temp +avg_temp +precipitation +timeid+green + ['city','total_cases','week_start_date']
features_selected= humid + dtr + max_temp +min_temp +avg_temp +precipitation +timeid+green  
# precipitation_amt_mm  & reanalysis_sat_precip_amt_mm are the same
 
panel = panel.loc[:,selected]
 

In [5]:
sj = panel.loc[panel.city == 'sj'].interpolate()  .set_index('week_start_date').copy().drop(['city'],axis = 1)
iq = panel.loc[panel.city == 'iq']  .interpolate()  .set_index('week_start_date').copy().iloc[60:].drop(['city'],axis = 1)
 

In [6]:
city_name = 'iq'

iq_test = FeatureTest.loc[FeatureTest.city == city_name].interpolate().set_index('week_start_date').copy().drop('city',axis = 1)
iq_test['total_cases'] = np.nan 

iq = pd.concat([iq,iq_test ],sort = False)
 
train_len = len(iq_test['total_cases'])

 
data_selected = iq.loc[:,['total_cases']] .copy() 
data_candidate = iq .copy().iloc[: ]
data_candidate['green_n'] = data_candidate.loc[:,green[:2]].mean(1)
data_candidate['green_s'] = data_candidate.loc[:,green[2:]].mean(1)

 


In [7]:
 

data_interaction = data_candidate.copy().drop(['weekofyear','year'],axis = 1)
def append_shift(left,right,selected,shift,smooth =52):
    new_names = []
    for name in selected:
        new_name = name + '_shift_' +str(shift)+'_smooth_' + str(smooth)
        new_names.append(new_name)
 
        left[new_name] = right.rolling(smooth  ).mean().shift(shift).loc[:,name ]
    return left

def append_diff(left,right,selected,diff,smooth =52):
    new_names = []
    for name in selected:
        new_name = name + '_diff_' +str(diff)+'_smooth_' + str(smooth)
        new_names.append(new_name)
 
        left[new_name] = right.rolling(smooth  ).mean().diff(diff).loc[:,name ]
    return left

 
def seasonal_predict(data,params):
    data = data.copy()
    data["season_sin"] = np.sin(data.index.weekofyear/53*6.2831)
    data["season_cos"] = np.cos(data.index.weekofyear/53*6.2831)
    
    expr = """  ~ """
    expr = 'total_cases' + expr
    
    for name in ["season_sin","season_cos"]:
        expr +=     name  + ' + '
    expr = expr[:-2]
   
    Y , X  = dmatrices(expr, data, return_type='dataframe')
    
    return  X.dot(params) 
 

def de_season_tri(data,target):
    data = data.copy()
    data["season_sin"] = np.sin(data.index.weekofyear/53*6.2831)
    data["season_cos"] = np.cos(data.index.weekofyear/53*6.2831)
    expr = """  ~ """
    expr = target + expr
    
    for name in ["season_sin","season_cos"]:
        expr +=     name  + ' + '
    expr = expr[:-2]
 
    Y , X  = dmatrices(expr, data, return_type='dataframe')
    if target == 'total_cases':
        Y = Y.clip(0,50)
    model = sm.OLS(Y,X)
    results = model.fit()
    
 
 
    
    return  data.loc[:,target] - results .predict(X)  ,results.params
residuals,params = de_season_tri(data_candidate,'total_cases')
data_selected['season_based'] = seasonal_predict(data_selected.fillna(0) ,params) 
 
from sklearn.preprocessing import PolynomialFeatures

Interactions = PolynomialFeatures(degree = 2,include_bias = False)


data_interaction_temp = data_interaction.drop('total_cases',axis = 1)
interactions_val = Interactions .fit_transform(data_interaction_temp ) 
names = Interactions .get_feature_names(data_interaction_temp .columns)
interactions = pd.DataFrame(interactions_val ,columns = names ,index =data_interaction_temp .index )

interactions = interactions.rolling(20).mean().shift(2)
interactions['total_cases'] = data_selected.total_cases
 
data_selected = interactions


def normalize(df, how ):
    if how == 'z':
        return (df - df.mean())/(df.std())
    if how == 'r':
        return (df - df.min())/(df.max() - df.min())
    
    
    
    
    
    
    
   
    
    
    
    
    
    
data_selected_corr = data_selected .corr().total_cases.drop([ 'total_cases'])
selected = data_selected_corr  .loc[abs(data_selected_corr)>0.15].sort_values().index
tmp = data_selected.loc[:,selected ]
length = tmp.shape[1]
"""
row = 0
corr_limit = 0.95
while(row < length ):
    
    corrs = tmp.iloc[:int(len(tmp) ),:] .corr()
    col = row +1
    drops = []
    
    while(col < length ):
        if abs(corrs .iloc[row,col])>corr_limit:
            drops.append(tmp.columns[col])
        
        col += 1
    tmp .drop(drops,axis = 1,inplace = True)
    length = tmp.shape[1]
    row += 1
tmp = data_selected.loc[:,tmp.columns].dropna()
print(tmp.shape)
"""

'\nrow = 0\ncorr_limit = 0.95\nwhile(row < length ):\n    \n    corrs = tmp.iloc[:int(len(tmp) ),:] .corr()\n    col = row +1\n    drops = []\n    \n    while(col < length ):\n        if abs(corrs .iloc[row,col])>corr_limit:\n            drops.append(tmp.columns[col])\n        \n        col += 1\n    tmp .drop(drops,axis = 1,inplace = True)\n    length = tmp.shape[1]\n    row += 1\ntmp = data_selected.loc[:,tmp.columns].dropna()\nprint(tmp.shape)\n'

In [8]:
data_selected= data_selected.loc[:,['total_cases']].merge(tmp,\
                                                           left_index = True,right_index = True)
 
data_selected.columns = [name.replace(' ','_') for name in data_selected.columns]
data_selected = data_selected.fillna(0)
data_selected['season_based'] = seasonal_predict(data_selected.fillna(0) ,params) 

In [9]:
data_selected.shape

(616, 160)

In [10]:

#data_temp = normalize( data_selected,'r')
#data_temp['total_cases'] =  data_selected.total_cases

#data_selected = data_temp.copy()



from sklearn.model_selection import KFold
data = data_selected.copy() .iloc[:-train_len ].dropna()
data_test = data_selected.copy().iloc[-train_len: ].fillna(method = 'ffill') .fillna(method = 'bfill').replace(np.nan,0)
kf = KFold(n_splits=4, random_state=None, shuffle=False)

expr = """total_cases ~   -1 + """
drops = ['total_cases'  ]
for name in data.drop(drops,axis = 1).columns:
    expr +=    name + ' + '
expr=  expr[:-2]

maes = []
models = []
test_xs = []
test_ys = []
pred_ys = []
 






def normalize(df, how ):
    if how == 'z':
        return (df - df.mean())/(df.std())
    if how == 'r':
        return (df - df.min())/(df.max() - df.min())


    
for train_index, test_index in kf.split(data):
    
    df_train = data.iloc[train_index,:]
    df_test  = data.iloc[test_index,:]
    
    #train_y,  train_X = dmatrices(expr, df_train, return_type='dataframe')
    train_y,train_X  = df_train.loc[:,['total_cases']],df_train.drop(['total_cases'],axis = 1)
    #test_y,  test_X = dmatrices(expr, df_test, return_type='dataframe')
    test_y,test_X  = df_test.loc[:,['total_cases']],df_test.drop(['total_cases'],axis = 1)
    test_xs.append(test_X)
    
    
   
    
    train_y = train_y.iloc[:,0] .clip(0,150).rolling(3,center = True).mean() .dropna()
    train_X = train_X.loc[train_y.index]
    """model"""
    model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=400,
             n_jobs=4, nthread=None, objective='reg:squarederror',
             random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)
    
    
    model .fit(X=train_X, y=pd.DataFrame(train_y))
    pred_res = pd.Series(model.predict(test_X)  ,index =  test_y .index)

    pred_res = pd.Series(pred_res,index =df_test.index )
    
    
    pred_ys.append(pred_res)
    test_ys.append(test_y)
    
    models.append(model)
    maes .append(abs(test_y.values.ravel()- pred_res.values.ravel()).mean())
    
 

In [11]:
test = pd.concat(test_ys) 
validate =pd.DataFrame( pd.concat(pred_ys)) 

ensamble_samples = validate.merge(test,left_index= True,right_index = True )
ensamble_samples.to_csv('./data/ensamble_xgb_' +str(city_name) + '.csv')

In [12]:
maes

[8.197260879433673, 10.357853977576546, 9.398311119494231, 8.759959931995558]

In [13]:
m1  = pd.Series(models[0].feature_importances_,index= train_X.columns).sort_values().iloc[-100:]
m2  = pd.Series(models[1].feature_importances_,index= train_X.columns).sort_values().iloc[-100:] 
m3  = pd.Series(models[2].feature_importances_,index= train_X.columns).sort_values().iloc[-100:]
m4  = pd.Series(models[3].feature_importances_,index= train_X.columns).sort_values().iloc[-100:]

In [14]:
df_total  = data 
    
total_y,  total_X = df_total .loc[:,['total_cases']],df_total .drop(['total_cases'],axis = 1)
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=400,
             n_jobs=4, nthread=None, objective='reg:squarederror',
             random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)
model.fit( total_X ,total_y.iloc[:,[0]].clip(0,150))

 

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=400,
             n_jobs=4, nthread=None, objective='reg:squarederror',
             random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [15]:
 
submit_X = data_test  .drop(['total_cases'],axis = 1)

pd.Series(np.round(model.predict(submit_X)).astype(int),index =  submit_X .index).\
to_csv("./data/0127_xgb_" + str(city_name) + ".csv",header = False)

In [16]:
clf.best_estimator_

NameError: name 'clf' is not defined

### Based model